In [1]:
import sys,os
add_path = os.path.abspath('../python')
if not add_path in sys.path:
    sys.path.append(add_path)
print(sys.path.index(add_path))

15


In [2]:
import time
import numpy as np
from numpy import vstack, hstack
from segbench import csaAssign, correspondPixels, matchEdgeMaps2D, matchEdgeMaps3D

In [8]:
data = np.array([[1, 4, 3],
    [2, 5, 3],
    [3, 6, 3],
    [1, 5, 1],
    [2, 6, 1],
    [3, 4, 5]])
n = 6
e1 = csaAssign(n,data)
print e1

[[ 1.  5.  1.]
 [ 2.  6.  1.]
 [ 3.  4.  5.]]


In [9]:
# Unit test for csaAssign.
n = 1000
dg = np.random.rand(n,n)>.5
m = np.sum(dg.flat)
i = np.array(dg.ravel().nonzero()).astype(float)
g1 = 1+np.floor(np.divide(i,n))
g2 = (1+np.floor(np.divide(i,n)),
     1+ np.mod(i,n)+n,
     1+np.floor(np.random.rand(1,m))*1000)
g2 = np.array(np.vstack(g2)).T
t = time.time()
e2 = csaAssign(n*2,g2)
print '{:.3e}s'.format(time.time() - t)
esum = np.sum(e2,axis=0)
if not all(esum[0:2]==(n*(n+1)/2,n*(n+1)/2+n*n)): raise ValueError("bug")
if not np.sum(esum[0:2])==(2*n*(2*n+1)/2): raise ValueError("bug")
print n,m, e2[3][:]

5.273e-02s
1000 500250 [  4.00000000e+00   1.00500000e+03   1.00000000e+00]


In [11]:
# Unit test for correspondPixels.
for n in np.round(np.logspace(0,2,10)).astype(int):
    for thresh in np.linspace(0,1,10):
        print 'n={:d} thresh={:1.2e}'.format(n,thresh),
        bmap1 = (np.random.rand(n,n)>thresh).astype(float)
        bmap2 = (np.random.rand(n,n)>thresh).astype(float)
        t = time.time()
        cost,oc,match1,match2 = correspondPixels(bmap1,bmap2,.005)
        print '{:1.2e}s'.format(time.time() - t),
        npix = len(bmap1.nonzero()[0]) + len(bmap2.nonzero()[0])
        ncor = len(match1.nonzero()[0]) + len(match1.nonzero()[0])
        nout = len(((bmap1>0)&(match1==0)).nonzero()[0]) + len(((bmap2>0)&(match2==0)).nonzero()[0])
        print npix,ncor,nout
        if ncor+nout!=npix: raise ValueError("bug")

        match1f = match1.ravel()
        idx1 = match1f.astype(int)
        a1 = match1f.nonzero()[0]
        a2 = match1f[idx1.nonzero()[0]].ravel()
        a = vstack((a1,a2)).T

        match2f = match2.ravel()
        idx2 = match2f.astype(int)
        b1 = match2f.nonzero()[0]
        b2 = match2f[idx2.nonzero()[0]].ravel()
        b = vstack((b2,b1)).T
        b = b[b[:,0].argsort(),]
        if np.sum(a-b): raise ValueError("bug")

n=1 thresh=0.00e+00 4.29e-05s 2 2 0
n=1 thresh=1.11e-01 3.60e-05s 2 2 0
n=1 thresh=2.22e-01 2.38e-05s 2 2 0
n=1 thresh=3.33e-01 2.29e-05s 2 2 0
n=1 thresh=4.44e-01 1.22e-05s 1 0 1
n=1 thresh=5.56e-01 2.81e-05s 1 0 1
n=1 thresh=6.67e-01 1.31e-05s 1 0 1
n=1 thresh=7.78e-01 1.22e-05s 0 0 0
n=1 thresh=8.89e-01 1.00e-05s 0 0 0
n=1 thresh=1.00e+00 9.06e-06s 0 0 0
n=2 thresh=0.00e+00 4.79e-05s 8 8 0
n=2 thresh=1.11e-01 4.51e-05s 8 8 0
n=2 thresh=2.22e-01 4.89e-05s 8 8 0
n=2 thresh=3.33e-01 2.60e-05s 4 4 0
n=2 thresh=4.44e-01 2.50e-05s 6 4 2
n=2 thresh=5.56e-01 2.10e-05s 4 2 2
n=2 thresh=6.67e-01 1.00e-05s 2 0 2
n=2 thresh=7.78e-01 1.00e-05s 0 0 0
n=2 thresh=8.89e-01 1.00e-05s 2 0 2
n=2 thresh=1.00e+00 1.00e-05s 0 0 0
n=3 thresh=0.00e+00 1.10e-04s 18 18 0
n=3 thresh=1.11e-01 1.05e-04s 18 18 0
n=3 thresh=2.22e-01 7.20e-05s 15 12 3
n=3 thresh=3.33e-01 4.39e-05s 10 6 4
n=3 thresh=4.44e-01 2.69e-05s 10 4 6
n=3 thresh=5.56e-01 2.60e-05s 9 4 5
n=3 thresh=6.67e-01 1.00e-05s 3 0 3
n=3 thresh=7.78e-01 

In [13]:
# Unit test for matchEdgeMaps2D.
for n in np.round(np.logspace(0,2,10)).astype(int):
    for thresh in np.linspace(0,1,10):
        print 'n={:d} thresh={:1.2e}'.format(n,thresh),
        bmap1 = (np.random.rand(n,n)>thresh).astype(float)
        bmap2 = (np.random.rand(n,n)>thresh).astype(float)
        t = time.time()
        cost,match1,match2 = matchEdgeMaps2D(bmap1,bmap2,.005,100)
        print '{:1.2e}s'.format(time.time() - t),
        npix = len(bmap1.nonzero()[0]) + len(bmap2.nonzero()[0])
        ncor = len(match1.nonzero()[0]) + len(match1.nonzero()[0])
        nout = len(((bmap1>0)&(match1==0)).nonzero()[0]) + len(((bmap2>0)&(match2==0)).nonzero()[0])
        print ncor,'+',nout,'=',ncor+nout,'==',npix
        if ncor+nout!=npix: raise ValueError("bug")
        
        match1f = match1.ravel()
        idx1 = match1f.astype(int)
        a1 = match1f.nonzero()[0]
        a2 = match1f[idx1.nonzero()[0]].ravel()
        a = vstack((a1,a2)).T

        match2f = match2.ravel()
        idx2 = match2f.astype(int)
        b1 = match2f.nonzero()[0]
        b2 = match2f[idx2.nonzero()[0]].ravel()
        b = vstack((b2,b1)).T
        b = b[b[:,0].argsort(),]
        if np.sum(a-b): raise ValueError("bug")

n=1 thresh=0.00e+00 4.60e-05s 2 + 0 = 2 == 2
n=1 thresh=1.11e-01 1.60e-05s 0 + 1 = 1 == 1
n=1 thresh=2.22e-01 1.91e-05s 0 + 1 = 1 == 1
n=1 thresh=3.33e-01 3.91e-05s 2 + 0 = 2 == 2
n=1 thresh=4.44e-01 2.10e-05s 0 + 1 = 1 == 1
n=1 thresh=5.56e-01 2.10e-05s 0 + 0 = 0 == 0
n=1 thresh=6.67e-01 2.19e-05s 0 + 1 = 1 == 1
n=1 thresh=7.78e-01 1.91e-05s 0 + 1 = 1 == 1
n=1 thresh=8.89e-01 1.98e-05s 0 + 0 = 0 == 0
n=1 thresh=1.00e+00 2.00e-05s 0 + 0 = 0 == 0
n=2 thresh=0.00e+00 6.60e-05s 8 + 0 = 8 == 8
n=2 thresh=1.11e-01 3.10e-05s 4 + 1 = 5 == 5
n=2 thresh=2.22e-01 3.70e-05s 6 + 0 = 6 == 6
n=2 thresh=3.33e-01 3.00e-05s 4 + 2 = 6 == 6
n=2 thresh=4.44e-01 1.10e-05s 0 + 4 = 4 == 4
n=2 thresh=5.56e-01 3.79e-05s 2 + 1 = 3 == 3
n=2 thresh=6.67e-01 3.00e-05s 4 + 1 = 5 == 5
n=2 thresh=7.78e-01 1.69e-05s 0 + 0 = 0 == 0
n=2 thresh=8.89e-01 2.31e-05s 0 + 2 = 2 == 2
n=2 thresh=1.00e+00 2.79e-05s 0 + 0 = 0 == 0
n=3 thresh=0.00e+00 1.90e-04s 18 + 0 = 18 == 18
n=3 thresh=1.11e-01 1.41e-04s 18 + 0 = 18 == 18
n=3 

In [16]:
# Unit test for matchEdgeMaps3D
for n in np.round(np.logspace(0,6,4,base=2)).astype(int):
    for thresh in np.linspace(.5,.9,4):
        print 'n={:d} thresh={:1.2e}'.format(n,thresh),
        bmap1 = (np.random.rand(n,n,n)>thresh).astype(float)
        bmap2 = (np.random.rand(n,n,n)>thresh).astype(float)

        t = time.time()
        cost,match1,match2 = matchEdgeMaps3D(bmap1,bmap2,.000005,100)
        print '{:1.4f}s'.format(time.time() - t),
        npix = len(bmap1.nonzero()[0]) + len(bmap2.nonzero()[0])
        ncor = len(match1.nonzero()[0]) + len(match1.nonzero()[0])
        nout = len(((bmap1>0)&(match1==0)).nonzero()[0]) + len(((bmap2>0)&(match2==0)).nonzero()[0])
        print ncor,'+',nout,'=',ncor+nout,'==',npix
        if ncor+nout!=npix: raise ValueError("bug")
        
        match1f = match1.ravel()
        idx1 = match1f.astype(int)
        a1 = match1f.nonzero()[0]
        a2 = match1f[idx1.nonzero()[0]].ravel()
        a = vstack((a1,a2)).T

        match2f = match2.ravel()
        idx2 = match2f.astype(int)
        b1 = match2f.nonzero()[0]
        b2 = match2f[idx2.nonzero()[0]].ravel()
        b = vstack((b2,b1)).T
        b = b[b[:,0].argsort(),]
        if np.sum(a-b): raise ValueError("bug")

n=1 thresh=5.00e-01 0.0000s 0 + 1 = 1 == 1
n=1 thresh=6.33e-01 0.0000s 0 + 1 = 1 == 1
n=1 thresh=7.67e-01 0.0000s 0 + 0 = 0 == 0
n=1 thresh=9.00e-01 0.0000s 0 + 0 = 0 == 0
n=4 thresh=5.00e-01 0.0003s 22 + 38 = 60 == 60
n=4 thresh=6.33e-01 0.0002s 18 + 29 = 47 == 47
n=4 thresh=7.67e-01 0.0000s 4 + 24 = 28 == 28
n=4 thresh=9.00e-01 0.0000s 2 + 10 = 12 == 12
n=16 thresh=5.00e-01 0.0308s 2014 + 2065 = 4079 == 4079
n=16 thresh=6.33e-01 0.0176s 1098 + 1874 = 2972 == 2972
n=16 thresh=7.67e-01 0.0054s 440 + 1443 = 1883 == 1883
n=16 thresh=9.00e-01 0.0013s 76 + 762 = 838 == 838
n=64 thresh=5.00e-01 7.9337s 131094 + 131138 = 262232 == 262232
n=64 thresh=6.33e-01 2.7748s 70290 + 122430 = 192720 == 192720
n=64 thresh=7.67e-01 0.8190s 28628 + 93645 = 122273 == 122273
n=64 thresh=9.00e-01 0.1273s 5146 + 47371 = 52517 == 52517


In [3]:
# Unit test for correspondPixels.
n = 10
thresh = .5
print 'n={:d} thresh={:1.2e}'.format(n,thresh),
bmap1 = (np.random.rand(n,n)>thresh).astype(float)
bmap2 = (np.random.rand(n,n)>thresh).astype(float)
t = time.time()
cost,oc,match1,match2 = correspondPixels(bmap1,bmap2,.005)
print '{:1.2e}s'.format(time.time() - t),
npix = len(bmap1.nonzero()[0]) + len(bmap2.nonzero()[0])
ncor = len(match1.nonzero()[0]) + len(match1.nonzero()[0])
nout = len(((bmap1>0)&(match1==0)).nonzero()[0]) + len(((bmap2>0)&(match2==0)).nonzero()[0])
print npix,ncor,nout
if ncor+nout!=npix: raise ValueError("bug")

match1f = match1.ravel()
idx1 = match1f.astype(int)
a1 = match1f.nonzero()[0]
a2 = match1f[idx1.nonzero()[0]].ravel()
a = vstack((a1,a2)).T

match2f = match2.ravel()
idx2 = match2f.astype(int)
b1 = match2f.nonzero()[0]
b2 = match2f[idx2.nonzero()[0]].ravel()
b = vstack((b2,b1)).T
b = b[b[:,0].argsort(),]
if np.sum(a-b): raise ValueError("bug")

n=10 thresh=5.00e-01 6.77e-04s 98 48 50
